In [1]:
# Connect to my google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!mkdir forVideo
!mkdir makeVideo

In [3]:
# get every frame of my video

import cv2
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/self driving car/lane detection/20210304_150312.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/forVideo/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
def color_mask(image):
    min = np.array([140,130,130])
    max = np.array([255,255,255])
    mask = cv2.inRange(image,min,max)
    return mask     

In [5]:
def CannyEdge(image):
  #gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  blur = cv2.GaussianBlur(image, (5,5), 0)
  # detection of the edges
  cannyImage = cv2.Canny(image, 100, 200, L2gradient = True)
  return cannyImage

In [24]:
# image has to be gray scaled
def region_of_interest(image):
    height, width = image.shape
    triangle = np.array([[(200, height),(1600, height),(1000, 700)]], np.int32)
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, triangle, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [7]:
rho = 2
theta = np.pi/180
threshold = 100

In [8]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

In [9]:
# convert 2(slope,intercept) points line to 4(x1,y1,x2,y2)
def make_points(image, line_parameters):
    slope, intercept = line_parameters
    y1 = int(image.shape[0])
    y2 = int(y1*3/5)
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]

In [18]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return None
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    if not left_fit:
        left_line = [[0,0,0,0]]
    else:
        left_fit_average = np.average(left_fit, axis = 0)
        left_line = make_points(image, left_fit_average)
    if not right_fit:
        right_line = [[0,0,0,0]]
    else:
        right_fit_average = np.average(right_fit, axis = 0)
        right_line = make_points(image, right_fit_average)
    return np.array((left_line, right_line))

In [19]:
# take width and height of image
test = cv2.imread("/content/forVideo/frame0.jpg")
height, width, channels = test.shape
print(width, height)

1920 1080


1166 is the lane width

In [25]:
from pathlib import Path
from tqdm import tqdm
import os
images_path = "/content/forVideo"
for img_name in tqdm(os.listdir(images_path), total=len(os.listdir(images_path))):
    lines = 0
    averaged_lines = 0
    image_path = os.path.join(images_path, img_name)
    image = cv2.imread(image_path)
    masked = color_mask(image)
    cannyImage = CannyEdge(masked)
    cropped_Image = region_of_interest(cannyImage)
    lines = cv2.HoughLinesP(cropped_Image,rho, theta, threshold, np.array ([ ]), minLineLength=20, maxLineGap=5)
    averaged_lines = average_slope_intercept(image, lines)   
    left_line, right_line = averaged_lines
    if left_line[0][0]!=0 or right_line[0][0]!=0:
        if left_line[0][0]==0:
            left_line[0][0] = right_line[0][0] - 1166
        if right_line[0][0]==0:
            right_line[0][0] = left_line[0][0] +1166
    if ((right_line[0][0] + left_line[0][0])/2 > width/2 + 30):
        text = "TURN RIGHT"
    elif ((right_line[0][0] + left_line[0][0])/2 < width/2 - 30):
        text = "TURN LEFT"
    else:
        text = "GO STRAIGHT"

    line_image = display_lines(image, averaged_lines)
    combo_image = cv2.addWeighted(image, 0.8, line_image, 1, 1)

    cv2.putText(combo_image, text ,
	                        (600, 500), cv2.FONT_HERSHEY_SIMPLEX,
	                        3.0, (0, 255, 0), 5)
    #Save image
    cv2.imwrite(f"/content/makeVideo/{img_name}", combo_image)

100%|██████████| 188/188 [00:15<00:00, 12.01it/s]


In [26]:
# make my video
fps = 8.0
image_folder = '/content/makeVideo'
video_name = 'testi.mp4'
num_of_images = os.listdir(image_folder)
num_of_images = len(num_of_images)
images = []
for _ in range(0, num_of_images, 1):
    images.append(f"{image_folder}/frame{_}.jpg")


frame = cv2.imread(images[0])
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter(video_name, fourcc, fps, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [ ]:
stop

NameError: ignored

In [27]:
# move the video to any folder
!mv /content/testi.mp4 "/content/gdrive/MyDrive/self driving car/lane detection"

In [28]:
# clear my dir
directory = "/content/makeVideo"
for img in os.listdir(directory):
    curPath = f"{directory}/{img}"
    !rm $curPath